In [1]:
import torch
from modeling import *

In [2]:
model = ToyTransformer(128, 8, 8, 128, 1024, torch.device('cuda'), torch.bfloat16)

In [3]:
global_config['attn_backend'] = AttentionBackend.FlashAttentionTriton
a, b = model.forward(torch.ones((1, 1), dtype=torch.long).cuda())

In [4]:
model.forward(torch.ones((1, 1), dtype=torch.long).cuda(), kv_cache=b)

(tensor([[[-0.4766, -0.6875, -1.3047, -0.6367,  0.7695,  0.2061,  0.5977,
           -1.2188,  1.6172, -1.0391,  0.0349,  1.5547,  0.0996,  0.2793,
            1.0547, -0.7344, -0.3145, -1.2266, -1.3984,  0.9297,  0.9648,
            1.1250, -0.1377, -1.2266, -0.3770,  0.1504,  0.4082,  0.5430,
           -1.4688, -0.9805,  0.8320, -0.3105,  0.6953, -0.3633,  0.3438,
           -1.0156, -0.4688,  0.8945,  0.8789,  0.3203,  0.4258,  0.2734,
           -0.1855, -0.0532, -0.4180,  0.1758, -0.7422, -0.0134,  0.9336,
            0.4102,  0.5391,  0.9336, -0.4160,  0.3340, -0.8438,  1.3750,
            1.1406, -0.6914, -0.1553, -0.5078, -0.0227, -0.3379, -0.1670,
            0.4297, -0.2793,  0.8242, -0.4277, -0.5781, -0.3184, -0.2422,
           -0.4121,  0.1982, -1.0469,  0.4473,  0.0272,  1.0547, -0.3789,
            0.0086,  1.1094, -0.6445, -1.1016,  1.7891,  0.2832, -0.2393,
           -1.3281,  0.4160, -1.0625, -0.4883,  1.1406, -2.0312,  0.6953,
           -0.9961,  0.7930, -0.4570, 